In [ ]:
# ======== COMMON CONFIG (run once) ==========================================
import os, sys, re, warnings, argparse, pandas as pd
warnings.filterwarnings("ignore")

from rolling_framework import Machine           # <-- 프로젝트의 핵심 API

# ---- 파일 경로 --------------------------------------------------------------
DATA_DIR      = "data/"
Y_FILE        = os.path.join(DATA_DIR, "exrets.csv")
SLOPE_FILE    = os.path.join(DATA_DIR, "slope.csv")
YL_FILE       = os.path.join(DATA_DIR, "yl_all.csv")
MACRO_FILE    = os.path.join(DATA_DIR, "MacroFactors.csv")

OUT_DIR       = "./output";  os.makedirs(OUT_DIR, exist_ok=True)

# ---- 샘플/예측 구간 ---------------------------------------------------------
BURN_START, BURN_END   = "197108", "199001"
PERIOD_START, PERIOD_END = "197108", "202312"
HORIZON = 12                           # months ahead

MATURITIES = ["xr_2","xr_3","xr_5","xr_7","xr_10"]

# ---- 유틸 함수 --------------------------------------------------------------
def _load_csv(path, name):
    try:  return pd.read_csv(path, index_col="Time")
    except FileNotFoundError as e:
        sys.exit(f"[ERROR] missing {name} → {e.filename}")

def _align_time(*dfs):
    idx=None
    for d in dfs: idx = d.index if idx is None else idx.intersection(d.index)
    return [d.loc[idx].sort_index() for d in dfs]

def _direct_pairs(slope_cols, y_cols):
    mk = lambda s: re.search(r"(\d+)", s).group(1) if re.search(r"(\d+)", s) else None
    y_map = {mk(c): c for c in y_cols}
    return [(sc, y_map[mk(sc)]) for sc in slope_cols if mk(sc) in y_map]

# ---- 데이터 로드 & 정렬 ------------------------------------------------------
y     = _load_csv(Y_FILE,   "exrets")
slope = _load_csv(SLOPE_FILE, "slope")
yl    = _load_csv(YL_FILE,   "yl_all")
macro = _load_csv(MACRO_FILE,"MacroFactors")

# 타깃 열 필터
y_cols = [c for c in MATURITIES if c in y.columns]
if not y_cols: sys.exit("[ERROR] MATURITIES not in exrets")
y = y[y_cols]

# 시간축 맞추기
y, slope, yl, macro = _align_time(y, slope, yl, macro)

# slope->y 자동 매핑  ex) slope_2 -> xr_2
DIRECT_PAIRS = _direct_pairs(slope.columns, y_cols)

print("✓ Loaded data shapes:",
      {k:v.shape for k,v in [("y",y),("slope",slope),("yl",yl),("macro",macro)]})
print("✓ direct map pairs :", DIRECT_PAIRS)

DNN_DUAL rolling:   4%|▍         | 20/520 [03:47<1:34:56, 11.39s/it]


In [ ]:
# ===== CASE : Base = slope(OLS), Residual = macro(MLP) =====================
# X 구성: slope + macro 모두 포함 (베이스/잔차가 같은 X에서 서로 다른 열을 사용)
X_macro = pd.concat([slope, macro], axis=1)

opt = {
    "base_on": True,                          # 베이스 켜기 → CS-Resi 형태
    "base_cols":   list(slope.columns),       # 베이스는 slope 만 사용
    "target_cols": list(y.columns),           # 예: ['xr_2','xr_3','xr_5','xr_7','xr_10']
    "residual_kind": "mlp",                   # 잔차학습은 단일 MLP
    "feature_cols": list(macro.columns),      # 잔차 입력은 macro 만 사용
    "standardize_res": True,                  # 잔차 입력만 표준화
    "mlp_hidden": (64, 32),
    "mlp_dropout": 0.1,
    "mlp_lr": 1e-3,
    "mlp_wd": 1e-4,
    "mlp_epochs": 200,
    "mlp_patience": 20,
    "seed": 0,
}

grid = {
    "arm__residual_model__module__hidden": [(64, 32), (128, 64)],
    "arm__residual_model__module__dropout": [0.0, 0.2],
    "arm__residual_model__optimizer__lr": [1e-3, 5e-4],
    "arm__residual_model__optimizer__weight_decay": [0.0, 1e-4],
}

m = Machine(
    X_macro, y, "ARM",
    option=opt, params_grid=grid,
    burn_in_start=BURN_START, burn_in_end=BURN_END,
    period=[PERIOD_START, PERIOD_END], forecast_horizon=HORIZON
)
print("\n▶ CASE  : base=slope(OLS), residual=macro-MLP")
m.training()
print(m.R2OOS())
print(m.MSEOOS())